# Lab 5: カスタマーサポートエージェントの GenAI Observability の詳細

## 概要

このLabでは、AgentCore Observability の仕組みと、AgentCore Runtime を使用せずにセットアップする方法を理解します。

## 追加する機能

**AgentCore Observability 機能**:
- Amazon OpenTelemetry Python Instrumentation の**セットアップ**  
- Amazon CloudWatch GenAI Observability でエージェントトレースを**可視化および分析**

## チュートリアルの詳細

| 情報 | 詳細 |
|-------------|---------|
| **チュートリアルタイプ** | 増分強化 |
| **エージェントタイプ** | 単一エージェント |
| **エージェントフレームワーク** | Strands Agents |
| **LLM モデル** | Anthropic Claude 3.7 Sonnet |
| **チュートリアルの業種** | カスタマーサポート |
| **複雑さ** | 簡単〜中程度 |
| **使用 SDK** | Strands SDK、AgentCore Observability、CloudWatch、Bedrock、boto3 |

## 前提条件

- **Lab 1 を先に完了する必要があります** - このLabは Lab 1 のエージェント上に直接構築されます 
- **Amazon CloudWatch でトランザクション検索を有効にする** - 初回ユーザーは Bedrock AgentCore のスパンとトレースを表示するために CloudWatch Transaction Search を有効にする必要があります。トランザクション検索を有効にするには、[ドキュメント](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) を参照してください。

## 学習目標

このLabを終了すると、以下ができるようになります：
- 公式の Amazon CloudWatch GenAI Observability ダッシュボードを使用する


---

## エージェントに Observability を追加しましょう

クライアントの初期化

In [ ]:
import boto3
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

logs_client = boto3.client("logs", region_name=region)
bedrock_client = boto3.client("bedrock", region_name=region)
sts_client = boto3.client("sts", region_name=region)

account_id = sts_client.get_caller_identity()["Account"]

`aws-opentelemetry-distro` がインストールされていることを確認してください

In [ ]:
%pip install strands-agents boto3 aws-opentelemetry-distro -q

# ステップ 2: Observability 用の環境設定

Strands エージェントのオブザーバビリティを有効にし、テレメトリデータを Amazon CloudWatch に送信するには、以下の環境変数を設定する必要があります。これらの設定を安全に管理するために `.env` ファイルを作成します。機密性の高い AWS 認証情報をコードから分離しながら、異なる環境間で簡単に切り替えられるようにします。

必要な環境変数:

| 変数 | 値 | 目的 |
|----------|-------|---------|
| `OTEL_PYTHON_DISTRO` | `aws_distro` | AWS Distro for OpenTelemetry (ADOT) を使用 |
| `OTEL_PYTHON_CONFIGURATOR` | `aws_configurator` | ADOT SDK 用の AWS コンフィギュレータを設定 |
| `OTEL_EXPORTER_OTLP_PROTOCOL` | `http/protobuf` | エクスポートプロトコルを設定 |
| `OTEL_TRACES_EXPORTER` | `otlp` | トレースエクスポータを設定 |
| `OTEL_EXPORTER_OTLP_LOGS_HEADERS` | `x-aws-log-group=<YOUR-LOG-GROUP>,x-aws-log-stream=<YOUR-LOG-STREAM>,x-aws-metric-namespace=<YOUR-NAMESPACE>` | ログを CloudWatch グループに送信 |
| `OTEL_RESOURCE_ATTRIBUTES` | `service.name=<YOUR-AGENT-NAME>` | オブザーバビリティデータでエージェントを識別 |
| `AGENT_OBSERVABILITY_ENABLED` | `true` | ADOT パイプラインを有効化 |

また、`AWS_REGION`、`AWS_DEFAULT_REGION`、`AWS_ACCOUNT_ID` 環境変数も設定してください。これらは opentelemetry instrument スクリプトによって使用されます。

In [ ]:
log_group_name = "agents/customer-support-assistant-logs"  # Your log group name
log_stream_name = "default"  # Your log stream name

# Create log group
try:
    logs_client.create_log_group(logGroupName=log_group_name)
    print(f"✅ Log group '{log_group_name}' created successfully")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceAlreadyExistsException":
        print(f"ℹ️  Log group '{log_group_name}' already exists")
    else:
        print(f"❌ Error creating log group: {e}")

# Create log stream
try:
    logs_client.create_log_stream(
        logGroupName=log_group_name, logStreamName=log_stream_name
    )
    print(f"✅ Log stream '{log_stream_name}' created successfully")
except ClientError as e:
    if e.response["Error"]["Code"] == "ResourceAlreadyExistsException":
        print(f"ℹ️  Log stream '{log_stream_name}' already exists")
    else:
        print(f"❌ Error creating log stream: {e}")

In [ ]:
# Create .env file
service_name = "customer-support-assistant-strands"

with open(".env", "w") as f:
    # AWS Configuration
    f.write(f"AWS_REGION={region}\n")
    f.write(f"AWS_DEFAULT_REGION={region}\n")
    f.write(f"AWS_ACCOUNT_ID={account_id}\n")

    # OpenTelemetry Configuration for AWS CloudWatch GenAI Observability
    f.write("OTEL_PYTHON_DISTRO=aws_distro\n")
    f.write("OTEL_PYTHON_CONFIGURATOR=aws_configurator\n")
    f.write("OTEL_EXPORTER_OTLP_PROTOCOL=http/protobuf\n")
    f.write("OTEL_TRACES_EXPORTER=otlp\n")
    f.write(
        f"OTEL_EXPORTER_OTLP_LOGS_HEADERS=x-aws-log-group={log_group_name},x-aws-log-stream={log_stream_name},x-aws-metric-namespace=agents\n"
    )
    f.write(f"OTEL_RESOURCE_ATTRIBUTES=service.name={service_name}\n")
    f.write("AGENT_OBSERVABILITY_ENABLED=true\n")

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Display the OTEL-related environment variables
otel_vars = [
    "OTEL_PYTHON_DISTRO",
    "OTEL_PYTHON_CONFIGURATOR",
    "OTEL_EXPORTER_OTLP_PROTOCOL",
    "OTEL_EXPORTER_OTLP_LOGS_HEADERS",
    "OTEL_RESOURCE_ATTRIBUTES",
    "AGENT_OBSERVABILITY_ENABLED",
    "OTEL_TRACES_EXPORTER",
]

print("OpenTelemetry 設定:\n")
for var in otel_vars:
    value = os.getenv(var)
    if value:
        print(f"{var}={value}")

# ステップ 3: Strands Agent の定義

次に、以前と同じエージェントを再定義しましょう。

トレースが作成されることを示すために、エージェントにシンプルな挨拶クエリを渡します。

また、セッション ID が登録されていることも確認します。

In [ ]:
!cp lab_helpers/lab1_strands_agent.py customer_support_agent.py

In [ ]:
%%writefile -a customer_support_agent.py

import os
import argparse
from boto3.session import Session
from opentelemetry import baggage, context
from lab_helpers.utils import get_ssm_parameter

from strands import Agent
from strands.models import BedrockModel


def parse_arguments():
    parser = argparse.ArgumentParser(description="Customer Support Agent")
    parser.add_argument(
        "--session-id",
        type=str,
        required=True,
        help="Session ID to associate with this agent run",
    )
    return parser.parse_args()


def set_session_context(session_id):
    """Set the session ID in OpenTelemetry baggage for trace correlation"""
    ctx = baggage.set_baggage("session.id", session_id)
    token = context.attach(ctx)
    print(f"セッション ID '{session_id}' をテレメトリコンテキストにアタッチしました")
    return token


def main():
    # Parse command line arguments
    args = parse_arguments()

    # Set session context for telemetry
    context_token = set_session_context(args.session_id)

    # Get region
    boto_session = Session()
    region = boto_session.region_name

    try:
        # Create the same basic agent from Lab 1
        MODEL = BedrockModel(
            model_id=MODEL_ID,
            temperature=0.3,
            region_name=region,
        )

        basic_agent = Agent(
            model=MODEL,
            tools=[
                get_product_info,
                get_return_policy,
            ],
            system_prompt=SYSTEM_PROMPT,
        )

        # Execute the travel research task
        query = """Greet the user and provide a financial advice."""

        result = basic_agent(query)
        print("結果:", result)

        print("✅ エージェントが正常に実行され、トレースが CloudWatch にプッシュされました")
    finally:
        # Detach context when done
        context.detach(context_token)


if __name__ == "__main__":
    main()

# ステップ 4: AWS OpenTelemetry Python Distro

環境が設定され、エージェントが作成されたので、オブザーバビリティがどのように機能するかを理解しましょう。[AWS OpenTelemetry Python Distro](https://pypi.org/project/aws-opentelemetry-distro/) は、コードの変更なしに Strands エージェントを自動的にインストルメント化し、テレメトリデータをキャプチャします。

このディストリビューションは以下を提供します:
- AgentCore Runtime の外部（EC2、Lambda など）でホストされている Strands Agent の**自動インストルメンテーション**
- シームレスな CloudWatch 統合のための **AWS 最適化設定**  

### インストルメント化されたエージェントの実行

Strands エージェントからトレースをキャプチャするには、Python を直接実行する代わりに `opentelemetry-instrument` コマンドを使用します。これにより、`.env` ファイルの環境変数を使用してインストルメンテーションが自動的に適用されます:

```bash
opentelemetry-instrument python customer_support_assistant_agent.py
```

このコマンドは以下を行います:

- .env ファイルから OTEL 設定を読み込む
- Strands、Amazon Bedrock 呼び出し、エージェントツールとデータベース、およびエージェントによる他のリクエストを自動的にインストルメント化
- トレースを CloudWatch に送信
- GenAI Observability ダッシュボードでエージェントの意思決定プロセスを可視化できるようにする

In [ ]:
!opentelemetry-instrument python customer_support_agent.py --session-id "session-1234"

# ステップ 5: Gen AI Observability での確認 

Observability を設定したので、AWS CloudWatch の GenAI Observability ダッシュボードでトレースを確認しましょう。CloudWatch - GenAI Observability - Bedrock AgentCore に移動します。

#### セッションビューページ:

![セッション](images/sessions_lab5_observability.png)

#### トレースビューページ:
![トレース](images/traces_lab5_observability.png)

## おめでとうございます！

**Strands エージェントで AgentCore Observability を実装しました**（AgentCore Runtime なし）！

### 達成したこと:

- **オブザーバビリティ**: テレメトリデータを Amazon CloudWatch に送信するように Strands エージェントを設定
- **セッション管理**: デバッグを容易にするためにセッションごとにトレースが保存されることを確認

## 次のステップ

さらに AgentCore 機能を追加する準備はできましたか？以下を続けてください:

- **Lab 6**: AgentCore Identity を使用して外部サービスと安全に認証

## リソース

- [AgentCore Observability ドキュメント](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html)
- [**公式 AgentCore Observability サンプル**](https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/01-tutorials/06-AgentCore-observability)

---

**素晴らしい仕事です！本番環境でカスタマーサポートエージェントのパフォーマンスをトレース、デバッグ、監視できるようになりました！**